In [2]:
import numpy as np
import pickle
import pandas as pd
import os
import matplotlib.pyplot as plt
import SimpleITK as sitk

In [67]:
def read_kImage(path):
    # Reads the image using SimpleITK
    itkimage = sitk.ReadImage(path)
    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = sitk.GetArrayFromImage(itkimage)
    return ct_scan

In [86]:
def generate_training_tables():
    basedir = "../1/train/ProstateX-TrainingLesionInformationv2/ProstateX-TrainingLesionInformationv2"
    findingsfile = "{}/{}".format(basedir,"ProstateX-Findings-Train.csv")
    imagesfile = "{}/{}".format(basedir,"ProstateX-Images-Train.csv")
    kimagesfile = "{}/{}".format(basedir,"ProstateX-Images-KTrans-Train.csv")
    #Generamos las 3 tablas básicas a partir de los archivos.cvs
    findings_table = pd.read_csv(findingsfile)
    images_table = pd.read_csv(imagesfile)
    kimages_table = pd.read_csv(kimagesfile)
    #Operación de inner join entre tablas.
    join_kimages_table = pd.merge(left=findings_table, right= kimages_table, how ="inner", on=["ProxID","fid","pos"])
    join_images_table =  pd.merge(left=findings_table, right=  images_table, how ="inner", on=["ProxID","fid","pos"])
    #agregamos una columna para insertar la imagen
    join_images_table["MRI"] = pd.Series(np.zeros((len(join_images_table),2,2,2)).tolist(), index= join_images_table.index)
    join_images_table["State"] = pd.Series(False, index= join_images_table.index)
    join_kimages_table["Kimg"] = pd.Series(np.zeros((len(join_kimages_table),2,2,2)).tolist(), index= join_kimages_table.index)
    join_kimages_table["State"] = pd.Series(False, index= join_kimages_table.index)
    return join_images_table, join_kimages_table

In [69]:
def extract_voxel_data(series_path):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames( series_path )
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    image_3d = sitk.GetArrayFromImage(image)
    return image_3d

In [93]:
def extract_slice_data(img_3d, ijk):
    state = False
    k_string = ijk.split()[2]
    max_bandwith = img_3d.shape[0]
    possible_k = int (k_string)
    if (possible_k >= max_bandwith):
        possible_k = max_bandwith -1
        slice_index = possible_k
        state = True
        print("The ijk requested is {}, there was an error since the shapes of the image is {}".format(ijk,str(img_3d.shape)))
    else:
        slice_index = possible_k
        print("The ijk requested is {},OK since shapes of the image is {}".format(ijk,str(img_3d.shape)))
    return state, img_3d[slice_index,:,:]

In [94]:
def set_MRI_value(index,image,state):
    join_images_table.at[index,"MRI"] = image
    join_images_table.at[index,"State"] = state

In [95]:
def set_kImg_value(index,image,state):
    join_kimages_table.at[index,"Kimg"] = image
    join_kimages_table.at[index,"State"] = state

In [88]:
def insert_k_images():
    for row in range(len(join_kimages_table)):
        path = "../1/Train/ProstateXKtrains-train-fixed"
        patient_id = join_kimages_table.ProxID.iloc[row]
        ijk = join_kimages_table.ijk.iloc[row]
        path = "{}/{}".format(path,patient_id)
        path = "{}/{}-Ktrans.mhd".format(path,patient_id)
        kimage_3d = read_kImage(path)
        state, kimage_2d = extract_slice_data(kimage_3d,ijk)
        set_kImg_value(row,kimage_2d,state)
        print("Se ha cargado la Kimagen2d #{} para el paciente {} ".format(row,patient_id))
        del path,patient_id,kimage_3d,state


In [97]:
def insert_MRI_images():
    for row in range (len(join_images_table)):
        patient_id = join_images_table.ProxID.iloc[row]
        series_name = join_images_table.DCMSerDescr.iloc[row]
        series_id = join_images_table.DCMSerNum.iloc[row]
        slice_ijk = join_images_table.ijk.iloc[row]
        path = "../1/Train/PROSTATEx/{}/".format(patient_id)#Selects the Patient ID
        path = "{}/{}".format(path,os.listdir(path)[0])#Selects the default folder inside the patient
        sequences = os.listdir(path) # Selects all the sequences
        sequence = [ sequence for sequence in sequences if sequence[0:len(str(series_id))] == str(series_id) ] [0]
        # Selects the only sequence that matches the condition.
        path = "{}/{}".format(path,sequence)
        image_3d = extract_voxel_data(path)
        state, image_2d = extract_slice_data(image_3d,slice_ijk)
        set_MRI_value(row,image_2d,state)
        print("Se ha cargado la imagen2d #{} para el paciente {} serie: {} ".format(row,patient_id,series_name))
        del path, patient_id, series_name, series_id,sequence,sequences, image_3d, image_2d

In [98]:
def get_training_testing_tables():
    number_of_cancer_samples=len(join_kimages_table[(join_kimages_table["ClinSig"] == True) & (join_kimages_table["zone"]=="PZ")])
    number_of_non_cancer_samples = len(join_kimages_table[(join_kimages_table["ClinSig"] == False) & (join_kimages_table["zone"]=="PZ")])
    print("We have {} cancer samples and {} non-cancer samples".format(number_of_cancer_samples, number_of_non_cancer_samples))
    training_cancer_samples = number_of_cancer_samples//2
    training_non_cancer_samples = number_of_non_cancer_samples//2
    training_cancer_samples_index = np.random.randint(0,number_of_cancer_samples-1,training_cancer_samples)
    training_non_cancer_samples_index = np.random.randint(0,number_of_non_cancer_samples-1,training_non_cancer_samples)
    training_cancer_samples_table = join_kimages_table[(join_kimages_table["ClinSig"] == True) & (join_kimages_table["zone"]=="PZ")].iloc[training_cancer_samples_index]
    training_non_cancer_samples_table = join_kimages_table[(join_kimages_table["ClinSig"] == False) & (join_kimages_table["zone"]=="PZ")].iloc[training_non_cancer_samples_index]
    #we get the posible samples that are not taken on the training set.
    samples = [i for i in range(training_cancer_samples)]
    test_cancer_samples_index = [sample for sample in samples if sample not in training_cancer_samples_index]
    test_cancer_samples_index = np.array(test_cancer_samples_index)
    samples = [i for i in range(training_non_cancer_samples)]
    test_non_cancer_samples_index = [sample for sample in samples if sample not in training_non_cancer_samples_index]
    test_non_cancer_samples_index = np.array(test_non_cancer_samples_index)
    test_cancer_samples_table = join_kimages_table[(join_kimages_table["ClinSig"] == True) & (join_kimages_table["zone"]=="PZ")].iloc[test_cancer_samples_index]
    test_non_cancer_samples_table = join_kimages_table[(join_kimages_table["ClinSig"] == False) & (join_kimages_table["zone"]=="PZ")].iloc[test_non_cancer_samples_index]
    del samples,test_cancer_samples_index,test_non_cancer_samples_index, training_cancer_samples_index
    training_table = training_cancer_samples_table.append(training_non_cancer_samples_table)
    testing_table = test_cancer_samples_table.append(test_non_cancer_samples_table)
    training_table = training_table.sample(frac=1).reset_index(drop=True)
    testing_table = testing_table.sample(frac=1).reset_index(drop=True)
    return training_table,testing_table

In [ ]:
join_images_table, join_kimages_table =  generate_training_tables()


In [101]:
insert_k_images()

The ijk requested is 56 75 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #0 para el paciente ProstateX-0000 
The ijk requested is 51 61 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #1 para el paciente ProstateX-0001 
The ijk requested is 52 73 11,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #2 para el paciente ProstateX-0002 
The ijk requested is 68 72 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #3 para el paciente ProstateX-0002 
The ijk requested is 80 69 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #4 para el paciente ProstateX-0003 
The ijk requested is 51 54 13,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #5 para el paciente ProstateX-0003 
The ijk requested is 74 56 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #6 para el paciente ProstateX-0004 
The ijk requested is 69 

The ijk requested is 47 67 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #72 para el paciente ProstateX-0046 
The ijk requested is 66 71 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #73 para el paciente ProstateX-0046 
The ijk requested is 58 82 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #74 para el paciente ProstateX-0047 
The ijk requested is 77 70 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #75 para el paciente ProstateX-0048 
The ijk requested is 71 76 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #76 para el paciente ProstateX-0049 
The ijk requested is 73 64 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #77 para el paciente ProstateX-0050 
The ijk requested is 53 72 9,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #78 para el paciente ProstateX-0051 
The ijk requested i

The ijk requested is 62 77 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #144 para el paciente ProstateX-0098 
The ijk requested is 77 79 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #145 para el paciente ProstateX-0099 
The ijk requested is 49 77 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #146 para el paciente ProstateX-0099 
The ijk requested is 54 51 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #147 para el paciente ProstateX-0100 
The ijk requested is 60 66 11,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #148 para el paciente ProstateX-0100 
The ijk requested is 62 69 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #149 para el paciente ProstateX-0100 
The ijk requested is 64 58 3,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #150 para el paciente ProstateX-0101 
The ijk requ

The ijk requested is 60 83 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #213 para el paciente ProstateX-0138 
The ijk requested is 58 63 3,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #214 para el paciente ProstateX-0138 
The ijk requested is 55 83 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #215 para el paciente ProstateX-0138 
The ijk requested is 58 70 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #216 para el paciente ProstateX-0139 
The ijk requested is 52 76 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #217 para el paciente ProstateX-0139 
The ijk requested is 62 78 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #218 para el paciente ProstateX-0139 
The ijk requested is 47 70 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #219 para el paciente ProstateX-0140 
The ijk reque

The ijk requested is 73 73 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #285 para el paciente ProstateX-0175 
The ijk requested is 69 62 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #286 para el paciente ProstateX-0176 
The ijk requested is 73 68 9,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #287 para el paciente ProstateX-0176 
The ijk requested is 51 59 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #288 para el paciente ProstateX-0177 
The ijk requested is 57 69 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #289 para el paciente ProstateX-0177 
The ijk requested is 63 56 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #290 para el paciente ProstateX-0177 
The ijk requested is 69 46 9,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #291 para el paciente ProstateX-0178 
The ijk requ

In [102]:
insert_MRI_images()

The ijk requested is 36 72 9,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen2d #0 para el paciente ProstateX-0000 serie: ep2d_diff_tra_DYNDIST_ADC 
The ijk requested is 36 72 9,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen2d #1 para el paciente ProstateX-0000 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 36 72 9,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen2d #2 para el paciente ProstateX-0000 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 36 72 9,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen2d #3 para el paciente ProstateX-0000 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 36 72 9,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen2d #4 para el paciente ProstateX-0000 serie: ep2d_diff_tra_DYNDISTCALC_BVAL 
The ijk requested is 139 166 12,OK since shapes of the image is (15, 320, 320)
Se ha cargado la imagen2d #5 para el paciente ProstateX-0000 serie: t2_tse_cor

The ijk requested is 168 165 3,OK since shapes of the image is (19, 320, 320)
Se ha cargado la imagen2d #51 para el paciente ProstateX-0003 serie: t2_tse_sag 
The ijk requested is 240 205 12,OK since shapes of the image is (21, 384, 384)
Se ha cargado la imagen2d #52 para el paciente ProstateX-0003 serie: t2_tse_tra 
The ijk requested is 80 69 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la imagen2d #53 para el paciente ProstateX-0003 serie: tfl_3d PD ref_tra_1.5x1.5_t3 
The ijk requested is 33 57 16,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen2d #54 para el paciente ProstateX-0003 serie: ep2d_diff_tra_DYNDIST_ADC 
The ijk requested is 33 57 16,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen2d #55 para el paciente ProstateX-0003 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 33 57 16,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen2d #56 para el paciente ProstateX-0003 serie: ep2d_diff_tra_DYNDIS

The ijk requested is 50 69 11,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen2d #101 para el paciente ProstateX-0006 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 50 69 11,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen2d #102 para el paciente ProstateX-0006 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 50 69 11,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen2d #103 para el paciente ProstateX-0006 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 50 69 11,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen2d #104 para el paciente ProstateX-0006 serie: ep2d_diff_tra_DYNDISTCALC_BVAL 
The ijk requested is 144 146 0,OK since shapes of the image is (3, 256, 256)
Se ha cargado la imagen2d #105 para el paciente ProstateX-0006 serie: t2_loc sag 
The ijk requested is 147 135 0,OK since shapes of the image is (13, 256, 256)
Se ha cargado la imagen2d #106 para el paciente ProstateX-0006 serie: t2_local

The ijk requested is 170 151 12,OK since shapes of the image is (19, 320, 320)
Se ha cargado la imagen2d #153 para el paciente ProstateX-0009 serie: t2_tse_cor 
The ijk requested is 143 155 9,OK since shapes of the image is (19, 320, 320)
Se ha cargado la imagen2d #154 para el paciente ProstateX-0009 serie: t2_tse_sag 
The ijk requested is 197 191 14,OK since shapes of the image is (21, 384, 384)
Se ha cargado la imagen2d #155 para el paciente ProstateX-0009 serie: t2_tse_tra 
The ijk requested is 66 64 11,OK since shapes of the image is (16, 128, 128)
Se ha cargado la imagen2d #156 para el paciente ProstateX-0009 serie: tfl_3d PD ref_tra_1.5x1.5_t3 
The ijk requested is 32 70 10,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen2d #157 para el paciente ProstateX-0010 serie: ep2d_diff_tra_DYNDIST_ADC 
The ijk requested is 32 70 10,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen2d #158 para el paciente ProstateX-0010 serie: ep2d_diff_tra_DYNDIST 

The ijk requested is 25 67 11,OK since shapes of the image is (63, 128, 84)
Se ha cargado la imagen2d #204 para el paciente ProstateX-0013 serie: ep2d_diff_tra_DYNDIST_MIX 
The ijk requested is 25 67 11,OK since shapes of the image is (63, 128, 84)
Se ha cargado la imagen2d #205 para el paciente ProstateX-0013 serie: ep2d_diff_tra_DYNDIST_MIX 
The ijk requested is 25 67 11,OK since shapes of the image is (63, 128, 84)
Se ha cargado la imagen2d #206 para el paciente ProstateX-0013 serie: ep2d_diff_tra_DYNDIST_MIX 
The ijk requested is 25 67 11,OK since shapes of the image is (21, 128, 84)
Se ha cargado la imagen2d #207 para el paciente ProstateX-0013 serie: ep2d_diff_tra_DYNDIST_MIXCALC_BVAL 
The ijk requested is 137 126 5, there was an error since the shapes of the image is (3, 256, 256)
Se ha cargado la imagen2d #208 para el paciente ProstateX-0013 serie: t2_loc sag 
The ijk requested is 148 125 1,OK since shapes of the image is (13, 256, 256)
Se ha cargado la imagen2d #209 para el pa

...

The ijk requested is 191 163 16,OK since shapes of the image is (23, 320, 320)
Se ha cargado la imagen2d #3654 para el paciente ProstateX-0201 serie: t2_tse_tra 
The ijk requested is 74 65 10,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3655 para el paciente ProstateX-0201 serie: tfl_3d dynamisch fast 
The ijk requested is 74 65 10,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3656 para el paciente ProstateX-0201 serie: tfl_3d dynamisch fast 
The ijk requested is 74 65 10,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3657 para el paciente ProstateX-0201 serie: tfl_3d dynamisch fast 
The ijk requested is 74 65 10,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3658 para el paciente ProstateX-0201 serie: tfl_3d dynamisch fast 
The ijk requested is 74 65 10,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3659 para el paciente ProstateX-0201 serie: tfl_3d

The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3704 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3705 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3706 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3707 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3708 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3709 para el paciente ProstateX-0202 serie: tfl

The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3757 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3758 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3759 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3760 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3761 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3762 para el paciente ProstateX-0202 serie: tfl

The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3807 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3808 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3809 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3810 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3811 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3812 para el paciente ProstateX-0203 serie: tfl

The ijk requested is 71 65 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3861 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 71 65 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3862 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 71 65 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3863 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 71 65 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3864 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 71 65 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3865 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 71 65 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3866 para el paciente ProstateX-0203 serie: tfl

In [134]:

MRI_problems_table = join_images_table[join_images_table["State"] == True] 
print("There were {} problems".format(len(MRI_problems_table)))
percent = 100*float(len(MRI_problems_table)/len(join_images_table))

print("It was the  {0:.5f}% of the dataset".format(percent))
display(MRI_problems_table[["ProxID","DCMSerDescr","ijk","State"]])


There were 88 problems
It was the  2.27449% of the dataset


,ProxID,DCMSerDescr,ijk,State
14,ProstateX-0001,t2_loc sag,126 145 3,True
27,ProstateX-0002,t2_loc sag,135 132 3,True
162,ProstateX-0010,t2_loc sag,129 154 4,True
208,ProstateX-0013,t2_loc sag,137 126 5,True
311,ProstateX-0022,t2_loc sag,141 135 4,True
321,ProstateX-0023,t2_loc sag,137 131 4,True
349,ProstateX-0025,ep2d_diff_tra_DYNDIST_ADC,61 66 25,True
357,ProstateX-0025,ep2d_diff_tra_DYNDISTCALC_BVAL,61 66 25,True
360,ProstateX-0025,t2_tse_cor,186 163 17,True
363,ProstateX-0025,t2_tse_tra,266 200 26,True


In [136]:
k_problems_table = join_kimages_table[join_kimages_table["State"] == True] 
print("There were {} problems".format(len(k_problems_table)))
percent = 100*float(len(k_problems_table)/len(join_kimages_table))

print("It was the  {0:.5f}% of the dataset".format(percent))
display(k_problems_table[["ProxID","ijk","State"]])

There were 3 problems
It was the  0.89552% of the dataset


,ProxID,ijk,State
34,ProstateX-0025,89 67 24,True
159,ProstateX-0105,63 78 17,True
248,ProstateX-0154,57 72 16,True


In [124]:
training_table, testing_table = get_training_testing_tables()

We have 36 cancer samples and 159 non-cancer samples
